In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm; tqdm.pandas()
pd.options.display.max_columns = 202
pd.options.display.max_rows = 300

In [2]:
train = pd.read_pickle('data/train_original.pkl')
test = pd.read_pickle('data/test_original.pkl')
special_cols = [col for col in train.columns if train[col].dtype != np.float64]
feature_cols = [col for col in train.columns if col not in special_cols]

In [3]:
all_df = pd.concat([train[feature_cols], test[feature_cols]], axis=0).reset_index(drop=True)
all_df['is_train'] = [1]*train.shape[0] + [0]*test.shape[0]
all_df['original_index'] = all_df.index

In [4]:
all_df = all_df.sort_values('var_68').reset_index(drop=True)

In [5]:
gp = all_df.groupby('var_68')

for col in tqdm(feature_cols):
    if col == 'var_68':
        continue
    all_df[col+'_gp_68_mean'] = all_df['var_68'].map(gp[col].mean())

100%|████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 49.42it/s]


In [6]:
new_feats = [c for c in all_df.columns if '_gp_68_mean' in c]
all_df[new_feats].iloc[:train.shape[0]].to_pickle('features/gp_time_feats_train.pkl')
all_df[new_feats].iloc[train.shape[0]:].reset_index(drop=True).to_pickle('features/gp_time_feats_test.pkl')